## Load Conversations

In [22]:
import sys
sys.path.append('.')

import pandas as pd
from utils import load_jsonl, dump_jsonl
# import numpy as np

In [23]:
def get_task1_conver(in_dir, col_label, skips=[]):
    conversations = load_jsonl(f"{in_dir}")
    
    
    def to_message_str(messages, users):
        s = []
        u = []
        for m in messages:
            if users[m['user_id']] == "SYS":
                s.append(m['text'])
            else:
                u.append(m['text'])
                
        return s, u
        
    newdata = []
    for row in conversations:
        row["messages"].sort(key=lambda x: x["date_created"], reverse=False)
        
        users = {}
        for m in row["messages"]:
            if m["user_id"] not in users:
#                 username = "USR"+str(len(users)+1) if len(users.keys())==0 else "SYS"
                username = "USR" if len(users.keys())==0 else "SYS"
                users[m["user_id"]] = username
                
#         if len(users)>2:
#             print("More than 1 users", len(users))
        
        
        messages = row["messages"]
        chunk_size = 100
        for i in range(0, len(messages), chunk_size):
            sub_messages = messages[i:i+chunk_size]
            s, u = to_message_str(sub_messages, users)
            
            if pd.isna(row[col_label]):
                continue
            
            if row[col_label] in skips:
                continue
                
            label = row[col_label]
                
            newdata.append({
                "user": u,
                "sys": s,
                "label": label,
                "nturn": len(sub_messages)
            })
        
#     n_val = int(len(newdata)*0.05)
#     n_test = n_val
    
#     test = newdata[0:n_test]
#     val = newdata[n_test:n_test+n_val]
#     train = newdata[n_test+n_val:]
    
    return pd.DataFrame(newdata)

In [24]:
df = get_task1_conver("./Task1/annotated_conersations.jsonl", "closeness", skips = [])
df.head()

Loaded 1234 records from ./Task1/annotated_conersations.jsonl


,user,sys,label,nturn
0,"[ดีๆวาย, เป็นยังไงบ้างช่วงนี้, รวยๆ เฮงๆ , เหม...","[ว่าไงปราง, เหมือนเดิม ขายของ นอนตื่น ขายของ ช...",1. Close,35
1,"[วันนี้เรามาพูดถึงเรื่องภาพยนตร์กัน, ปกฅิแล้วค...","[โอเคค่ะ, ที่บ้านค่ะ, ดูแทบทุกวันเลย, ดูได้ทุก...",1. Close,39
2,"[เธอ, สรุป อ้วนจะซื้อรถเมื่อไหร่, แล้วอ้วนดูรถ...","[อะไรหมู, กลางปี, ว่าจะเอา ativ ตัวใหม่, ว่าจ...",1. Close,40
3,"[มึง, แมวมึงเป็นไงกันบ้างอ่ะ, ตอนนี้มีกี่ตัวนะ...","[ว่า, เด็กๆหรอ, ซนปกติเลยมึง, 5ตัว, แค่บ้านกูน...",1. Close,35
4,"[อ้วน, อยากได้แมว, อยากเลี้ยง, เอาไว้ตอนตัวไม่...","[ไรหมู, เอาไปทำไมแมว, จะหาจากไหน, เดี๋ยวถามให้...",1. Close,39


In [25]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Close,551,551,551
2. Know each other,230,230,230
3. Don't know each other,435,435,435
4. Don't like each other,5,5,5


In [26]:
# df.groupby("label").count()

In [27]:
def get_task2_conver(in_dir, col_label, skips=[]):
    conversations = load_jsonl(f"{in_dir}")
        
    newdata = []
    for row in conversations:
        row["messages"].sort(key=lambda x: x["created_at"], reverse=False)
        
        users = {}
        for m in row["messages"]:
            if m["user_id"] not in users:
#                 username = "USR"+str(len(users)+1) if len(users.keys())==0 else "SYS"
                username = "USR" if len(users.keys())!=0 else "SYS"
                users[m["user_id"]] = username
                
        if len(users)>2:
            print("More than 1 users", len(users))
        
        
        
        messages = row["messages"]
        s = []
        u = []
        for m in messages:
            text = m['text'].replace("[USR]", "").replace("[URL]", "URL")
            if users[m['user_id']] == "SYS":
                s.append(text)
            else:
                u.append(text)
        
        label = row[col_label]
            
        if pd.isna(label):
            continue
        
        if label in skips:
            continue
                
        newdata.append({
            "user": u,
            "sys": s,
            "label": label,
            "nturn": len(messages)
        })
                
            
    n_val = int(len(newdata)*0.1)
    n_test = n_val
    
    test = newdata[0:n_test]
    val = newdata[n_test:n_test+n_val]
    train = newdata[n_test+n_val:]
    
    return pd.DataFrame(newdata)

In [28]:
df = get_task2_conver("./Task2/annotated/annotated.jsonl", "authority")
df.head()

Loaded 2486 records from ./Task2/annotated/annotated.jsonl


,user,sys,label,nturn
0,[ แตงโม ชวนแฟนไปค่ะ ไม่ใช่กระติกชวนไป กลับไปหา...,[ทบทวน\nงานคืนนั้น #กระติก ชวนแฟนพี่ตม.ไปด้วยน...,3. Not respect,3
1,[ รูปนี้จ้า ยัยน้องตัวเล็กของพี่มิว☀️🌻\n \n#...,[เนื่ิองจากวันนี้...เป็นวันครบรอบ 3 ปี\nมาเล่น...,1. Respect,3
2,[ แอบคิดเหมือนกัน้ลยค่ะว่าจะมีอีกรอบ],[เธรดนี้จะมาวิเคราะห์น้ำท่วมปี54 ที่จริงมันอาจ...,2. Normal,3
3,[ เนี่ยสิ่งที่กูพูด URL],[มาดิ้นควยไรชองมึง\nมึงอ่ะโชว์โง่ไอ้สัตว์ เสร่...,3. Not respect,3
4,"[ สรุปปวินนี่บล็อกคนด้วยมั้ย?, สรุป ที่น่าโมโ...",[แต่ปวินกับไพรวัลย์ก็ไม่เข้าใจนะเหมือนเจ้เกลีย...,2. Normal,5


In [29]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Respect,319,319,319
2. Normal,1661,1661,1661
3. Not respect,364,364,364


In [30]:
df = get_task1_conver("./Task3/annotated/annotated.jsonl", "closeness")
df.head()

Loaded 1221 records from ./Task3/annotated/annotated.jsonl


,user,sys,label,nturn
0,"[ดีๆวาย, เป็นยังไงบ้างช่วงนี้, รวยๆ เฮงๆ , เหม...","[ว่าไงปราง, เหมือนเดิม ขายของ นอนตื่น ขายของ ช...",1. Close,35
1,"[วันนี้เรามาพูดถึงเรื่องภาพยนตร์กัน, ปกฅิแล้วค...","[โอเคค่ะ, ที่บ้านค่ะ, ดูแทบทุกวันเลย, ดูได้ทุก...",2. Know each other,39
2,"[เธอ, สรุป อ้วนจะซื้อรถเมื่อไหร่, แล้วอ้วนดูรถ...","[อะไรหมู, กลางปี, ว่าจะเอา ativ ตัวใหม่, ว่าจ...",2. Know each other,40
3,"[มึง, แมวมึงเป็นไงกันบ้างอ่ะ, ตอนนี้มีกี่ตัวนะ...","[ว่า, เด็กๆหรอ, ซนปกติเลยมึง, 5ตัว, แค่บ้านกูน...",1. Close,35
4,"[อ้วน, อยากได้แมว, อยากเลี้ยง, เอาไว้ตอนตัวไม่...","[ไรหมู, เอาไปทำไมแมว, จะหาจากไหน, เดี๋ยวถามให้...",1. Close,39


In [31]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Close,462,462,462
2. Know each other,696,696,696
3. Don't know each other,52,52,52
4. Don't like each other,11,11,11


## Naive Bayes

In [32]:
import pickle
def save_obj_values(filepath, obj):
  with open(filepath, 'wb') as fin:
    pickle.dump(obj, fin)
    
def load_obj_values(filepath):
  with open(filepath, 'rb') as fin:
    obj = pickle.load(fin)
  return obj

In [33]:
clse_print_labels = ['1. Close', '2. Know each other', "3. Don't know each other", "4. Don't like each other"]
auth_print_labels = ['0. Very respect', '1. Respect', '2. Normal', '3. Not respect']

In [39]:
import re
from itertools import groupby
import emoji
from pythainlp.util import countthai
from tqdm import tqdm

from pythainlp.tokenize import word_tokenize

def rm_reptitive(text):
    s = ""
    groups = groupby(text)
    for label, group in groups:
        g = list(group)
        if len(g) >= 3:
            s += f"{label*3} rep "
        else:
            s += "".join(g)
    return s

def remove_space(sent):
    newwords = []
    for w in sent:
        if len(w.strip())==0:
            continue
        newwords.append(w)
    return newwords

def analyse_conv_per_person(vocab, texts):    
    texts = [t.lower() for t in texts]
    texts = [rm_reptitive(t) for t in texts]
    words = [word_tokenize(t) for t in texts]
    words = [remove_space(w) for w in words]
    
    wc = {}
    for sents in words:
        for w in sents:
            if w not in vocab:
                vocab[w] = 0

            if w not in wc:
                wc[w] = 0
            
            vocab[w] += 1
            wc[w] += 1
    
    return vocab, wc


def analyse_conversation(df):
    wc = []
    vocab = {}
    labels = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        vocab, ms = analyse_conv_per_person(vocab, row["sys"])
        vocab, mu = analyse_conv_per_person(vocab, row["user"])
        wc.append((ms, mu))
        labels.append(row["label"])
    
    return vocab, wc, labels

In [52]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.naive_bayes import MultinomialNB

def get_data(wc, labels, feats, idxs):
    X = []
    Y = []
    for idx in idxs:
        _, mu = wc[idx]
        label = labels[idx]

        vec = []
        for w in feats:
            if w in mu:
                vec.append(mu[w])
            else:
                vec.append(0)
        X.append(vec)
        Y.append(label)

    return X, Y

def train_model(vocab, wc, labels, label_fn):
    sorted_tf = sorted(vocab.items(), key=lambda x:-x[1])
    
    feats = []
    for i, (k, v) in enumerate(sorted_tf):
        
        if i > 5000:    # use only top 5000
            break
        
        if v <= 5:     # use when tf >= 5
            break    
        feats.append(k)
    
    idxs = np.arange(0, len(labels), 1)
    np.random.shuffle(idxs)
    
    train_idx, test_idx = np.split(idxs, [int(0.9*len(idxs))])
    Xtrain, Ytrain = get_data(wc, labels, feats, train_idx)
    Xtest, Ytest = get_data(wc, labels, feats, test_idx)
    

    clf = MultinomialNB(force_alpha=True)
    clf.fit(Xtrain, Ytrain)
    
    predictions = clf.predict(Xtest)
    
    true_labels_value = []
    predictions_value = []
    
    for l, p in zip(Ytest, predictions):
        if label_fn(l) is None:
            continue
            
        if label_fn(p) is None:
            print(p)
            
        true_labels_value.append(label_fn(l))
        predictions_value.append(label_fn(p))
        
    
    p, r, f1, _ = precision_recall_fscore_support(Ytest, predictions, average='macro', zero_division=0)
    return {
        "rmse": np.sqrt(mean_squared_error(true_labels_value, predictions_value)),
        "r2": r2_score(true_labels_value, predictions_value),
        
        "accuracy": accuracy_score(Ytest, predictions),
        "f1": f1,
        "precision": p,
        "recall": r,
    }

In [53]:
def print_accuracy(vocab, wc, labels, label_fn):
    eval_metrics = {}
    for i in tqdm(range(10)):
        metrics = train_model(vocab, wc, labels, label_fn)
        for k in metrics:
            if k not in eval_metrics:
                eval_metrics[k] = []
            eval_metrics[k].append(metrics[k])

    for k in eval_metrics:
        print(f"{k}: {np.mean(eval_metrics[k]):.3f} ± {np.std(eval_metrics[k]):.3f}")

In [54]:
def closeness_to_vec1(label):
    if label == "1. Close":
        return 1
    elif label == "2. Know each other":
        return 0.5
    elif label == "3. Don't know each other": 
        return 0
    elif label == "4. Don't like each other":
        return None
    else:
        return None

def authority_to_vec1(label):
    if label == '0. Very respect':
        return 1
    elif label == '1. Respect': 
        return 0.5
    elif label == '2. Normal':
        return 0
    elif label == '3. Not respect':
        return None
    else:
        return None
    
def authority_to_vec2(label):
    if label == '0. Very respect':
        return None
    elif label == '1. Respect': 
        return 1
    elif label == '2. Normal':
        return 0.5
    elif label == '3. Not respect':
        return 0
    else:
        return None
    
    

In [55]:
df = get_task1_conver("./Task1/annotated_conersations.jsonl", "closeness", skips = [])
vocab, wc, labels = analyse_conversation(df)

Loaded 1234 records from ./Task1/annotated_conersations.jsonl


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1221/1221 [00:03<00:00, 365.46it/s]


In [56]:
print_accuracy(vocab, wc, labels, closeness_to_vec1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.23it/s]

rmse: 0.337 ± 0.034
r2: 0.410 ± 0.134
accuracy: 0.684 ± 0.040
f1: 0.563 ± 0.077
precision: 0.560 ± 0.077
recall: 0.578 ± 0.074


In [60]:
df = get_task2_conver("./Task2/annotated/annotated.jsonl", "closeness", skips = ["4. Don't like each other"])
vocab, wc, labels = analyse_conversation(df)

Loaded 2486 records from ./Task2/annotated/annotated.jsonl


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1867/1867 [00:02<00:00, 728.82it/s]


In [61]:
print_accuracy(vocab, wc, labels, closeness_to_vec1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:12<00:00,  1.20s/it]

rmse: 0.394 ± 0.027
r2: -0.392 ± 0.175
accuracy: 0.766 ± 0.033
f1: 0.435 ± 0.051
precision: 0.480 ± 0.074
recall: 0.427 ± 0.042


In [67]:
df = get_task1_conver("./Task3/annotated/annotated.jsonl", "closeness", skips = ["4. Don't like each other"])
vocab, wc, labels = analyse_conversation(df)

Loaded 1221 records from ./Task3/annotated/annotated.jsonl


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1210/1210 [00:03<00:00, 354.33it/s]


In [68]:
print_accuracy(vocab, wc, labels, closeness_to_vec1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.11it/s]

rmse: 0.232 ± 0.028
r2: 0.326 ± 0.114
accuracy: 0.793 ± 0.037
f1: 0.542 ± 0.022
precision: 0.527 ± 0.024
recall: 0.559 ± 0.021


In [69]:
# def authority_to_vec1(label):
#     if label == '0. Very respect':
#         return 1
#     elif label == '1. Respect': 
#         return 0.5
#     elif label == '2. Normal':
#         return 0
#     elif label == '3. Not respect':
#         return None
#     else:
#         return None
    
# def authority_to_vec2(label):
#     if label == '0. Very respect':
#         return None
#     elif label == '1. Respect': 
#         return 1
#     elif label == '2. Normal':
#         return 0.5
#     elif label == '3. Not respect':
#         return 0
#     else:
#         return None

In [72]:
df = get_task1_conver("./Task1/annotated_conersations.jsonl", "authority", skips = ["3. Not respect"])
vocab, wc, labels = analyse_conversation(df)

Loaded 1234 records from ./Task1/annotated_conersations.jsonl


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1220/1220 [00:03<00:00, 369.81it/s]


In [73]:
print_accuracy(vocab, wc, labels, authority_to_vec1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.22it/s]

rmse: 0.479 ± 0.029
r2: -0.500 ± 0.232
accuracy: 0.552 ± 0.037
f1: 0.470 ± 0.046
precision: 0.479 ± 0.049
recall: 0.470 ± 0.042


In [74]:
df = get_task2_conver("./Task2/annotated/annotated.jsonl", "authority", skips = [])
vocab, wc, labels = analyse_conversation(df)

Loaded 2486 records from ./Task2/annotated/annotated.jsonl


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2344/2344 [00:03<00:00, 727.93it/s]


In [75]:
print_accuracy(vocab, wc, labels, authority_to_vec2)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:17<00:00,  1.74s/it]

rmse: 0.241 ± 0.015
r2: 0.195 ± 0.060
accuracy: 0.769 ± 0.028
f1: 0.678 ± 0.028
precision: 0.682 ± 0.026
recall: 0.680 ± 0.038


In [76]:
df = get_task1_conver("./Task3/annotated/annotated.jsonl", "authority", skips = [])
vocab, wc, labels = analyse_conversation(df)

Loaded 1221 records from ./Task3/annotated/annotated.jsonl


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1221/1221 [00:03<00:00, 368.61it/s]


In [77]:
print_accuracy(vocab, wc, labels, authority_to_vec2)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.21it/s]

rmse: 0.202 ± 0.022
r2: -0.106 ± 0.175
accuracy: 0.837 ± 0.037
f1: 0.535 ± 0.051
precision: 0.606 ± 0.092
recall: 0.543 ± 0.063
